Combines YAMNet feature extractor with best classifier
and exports as TensorFlow Lite model for deployment.

In [7]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [8]:
MODELS_DIR = '../models/models_approach1'
TFLITE_DIR = '../models/models_approach1/tflite'
RESULTS_DIR = '../results/results_approach1'
YAMNET_MODEL_URL = 'https://tfhub.dev/google/yamnet/1'
TARGET_SR = 16000
RANDOM_SEED = 42

os.makedirs(TFLITE_DIR, exist_ok=True)

print(f"\nTensorFlow version: {tf.__version__}")



TensorFlow version: 2.15.0


In [9]:
# LOAD TRAINED COMPONENTS

# Load label encoder
label_encoder = joblib.load(os.path.join(MODELS_DIR, 'label_encoder.pkl'))
classes = label_encoder.classes_
print(f"\nLoaded label encoder")
print(f"  Classes: {', '.join(classes)}")

# Load scaler
scaler = joblib.load(os.path.join(MODELS_DIR, 'feature_scaler.pkl'))
print(f"Loaded feature scaler")

# Determine best model from comparison results
comparison_df = pd.read_csv(os.path.join(RESULTS_DIR, 'model_comparison.csv'))
best_model_name = comparison_df.sort_values('Test F1', ascending=False).iloc[0]['Model']
print(f"\nBest model identified: {best_model_name}")

# Load best model
model_path = os.path.join(MODELS_DIR, f'{best_model_name.lower()}_model.pkl')
best_classifier = joblib.load(model_path)
print(f"Loaded best classifier from: {model_path}")

# Load YAMNet
print(f"\nLoading YAMNet model...")
yamnet_model = hub.load(YAMNET_MODEL_URL)
print(f"YAMNet loaded successfully")



Loaded label encoder
  Classes: Alarm_Clock, Car_Horn, Glass_Breaking, Gunshot, Siren
Loaded feature scaler

Best model identified: XGBoost
Loaded best classifier from: ../models/models_approach1\xgboost_model.pkl

Loading YAMNet model...
YAMNet loaded successfully


In [10]:
# BUILD TENSORFLOW MODEL WRAPPER
print("BUILDING TENSORFLOW WRAPPER")

class YAMNetClassifierModel(tf.Module):
    """
    Complete pipeline: Audio → YAMNet → Scaler → Classifier → Predictions
    
    IMPORTANT: This model expects audio input of exactly 0.96 seconds (15,360 samples at 16kHz)
    For variable-length audio, preprocess into 0.96s segments before inference.
    """
    def __init__(self, yamnet_model, scaler, classifier, label_encoder):
        super().__init__()
        self.yamnet_model = yamnet_model
        self.scaler_mean = tf.constant(scaler.mean_, dtype=tf.float32)
        self.scaler_scale = tf.constant(scaler.scale_, dtype=tf.float32)
        self.classifier_type = type(classifier).__name__
        self.label_encoder = label_encoder
        self.class_names = tf.constant(label_encoder.classes_.tolist())
        
        # Store classifier parameters based on type
        if self.classifier_type == 'LogisticRegression':
            self.weights = tf.constant(classifier.coef_.T, dtype=tf.float32)
            self.bias = tf.constant(classifier.intercept_, dtype=tf.float32)
        else:
            # For other classifiers, we'll need to use them in numpy mode
            self.classifier = classifier
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[15360], dtype=tf.float32)])
    def __call__(self, audio):
        """
        Run inference on audio input
        
        Args:
            audio: tf.Tensor of shape (15360,) - 0.96s at 16kHz
            
        Returns:
            Dictionary with:
                - probabilities: (5,) probability for each class
                - predicted_class_id: scalar int
                - predicted_class_name: string class name
                - confidence: scalar float (max probability)
        """
        # Extract YAMNet embeddings
        scores, embeddings, spectrogram = self.yamnet_model(audio)
        features = tf.reduce_mean(embeddings, axis=0, keepdims=True)
        
        # Scale features
        features_scaled = (features - self.scaler_mean) / self.scaler_scale
        
        # Get predictions
        if self.classifier_type == 'LogisticRegression':
            # TF-only operations
            logits = tf.matmul(features_scaled, self.weights) + self.bias
            probabilities = tf.nn.softmax(logits)[0]
        else:
            # Use tf.py_function to wrap numpy operations
            def sklearn_predict(features):
                return self.classifier.predict_proba(features)[0].astype('float32')

            probabilities = tf.py_function(sklearn_predict, [features_scaled], tf.float32)

        
        predicted_class_id = tf.argmax(probabilities)
        predicted_class_name = self.class_names[predicted_class_id]
        confidence = tf.reduce_max(probabilities)
        
        return {
            'probabilities': probabilities,
            'predicted_class_id': predicted_class_id,
            'predicted_class_name': predicted_class_name,
            'confidence': confidence
        }

# Build model
print("\nBuilding full pipeline model...")
full_model = YAMNetClassifierModel(yamnet_model, scaler, best_classifier, label_encoder)
print("Model wrapper created")

# Test the model
print("\nTesting model wrapper...")
test_audio = tf.random.normal([15360], dtype=tf.float32)
test_output = full_model(test_audio)

print(f"Model test passed:")
print(f"  Input shape: {test_audio.shape}")
print(f"  Output probabilities shape: {test_output['probabilities'].shape}")
print(f"  Predicted class ID: {test_output['predicted_class_id'].numpy()}")
print(f"  Predicted class name: {test_output['predicted_class_name'].numpy().decode('utf-8')}")
print(f"  Confidence: {test_output['confidence'].numpy():.4f}")

BUILDING TENSORFLOW WRAPPER

Building full pipeline model...
Model wrapper created

Testing model wrapper...
Model test passed:
  Input shape: (15360,)
  Output probabilities shape: (5,)
  Predicted class ID: 3
  Predicted class name: Gunshot
  Confidence: 0.6399


In [11]:
# SAVE FULL MODEL
saved_model_path = os.path.join(MODELS_DIR, 'full_pipeline_savedmodel')
tf.saved_model.save(full_model, saved_model_path)
print(f"Saved full model to: {saved_model_path}")

INFO:tensorflow:Assets written to: ../models/models_approach1\full_pipeline_savedmodel\assets


INFO:tensorflow:Assets written to: ../models/models_approach1\full_pipeline_savedmodel\assets


Saved full model to: ../models/models_approach1\full_pipeline_savedmodel
